# Data Science Challenge

In [2]:
# If you'd like to install packages that aren't installed by default, uncomment the last two lines of this cell and replace <package list> with a list of your packages.
# This will ensure your notebook has all the dependencies and works everywhere

#import sys
#!{sys.executable} -m pip install <package list>

In [3]:
#Libraries
import pandas as pd
pd.set_option("display.max_columns", 101)

## Data Description

Column | Description
:---|:---
`id` | Unique Identifier
`brand` | The brand name of the product
`capacity` | The volume of each container (lt/ml)
`city`| The name of the city for each shop 
`container`| The type (nature) of the container.
`date`| Date of sale
`lat`| The latitude of the shop
`long`| The longitude of the shop
`pop` | Population of the city (assumed constant)
`shop` | the shop id
`price`| The average monthly price of the corresponding container (in US `$$`)
`quantity` | The number of containers sold in a month (target variable)

## Data Wrangling & Visualization

In [4]:
data = pd.read_csv('train.csv', index_col=0)

In [5]:
data.head()

,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
id,,,,,,,,,,,
0.0,31/01/12,Athens,37.97945,23.71622,672130.0,shop_1,kinder-cola,glass,500ml,0.96,13280.0
1.0,31/01/12,Athens,37.97945,23.71622,672130.0,shop_1,kinder-cola,plastic,1.5lt,2.86,6727.0
2.0,31/01/12,Athens,37.97945,23.71622,672130.0,shop_1,kinder-cola,can,330ml,0.87,9848.0
3.0,31/01/12,Athens,37.97945,23.71622,672130.0,shop_1,adult-cola,glass,500ml,1.00,20050.0
4.0,31/01/12,Athens,37.97945,23.71622,672130.0,shop_1,adult-cola,can,330ml,0.39,25696.0


In [6]:
#Explore columns
data.columns

Index(['date', 'city', 'lat', 'long', 'pop', 'shop', 'brand', 'container',
       'capacity', 'price', 'quantity'],
      dtype='object')

In [7]:
#Description
data.describe()

,lat,long,pop,price,quantity
count,6429.000000,6434.000000,6480.000000,6480.000000,6480.000000
mean,38.301238,23.270246,355141.416667,1.188981,29431.855093
std,1.649771,1.086960,232828.796289,0.814567,17901.328301
min,35.327870,21.734440,134219.000000,0.110000,2953.000000
25%,37.962450,22.417610,141439.750000,0.610000,16441.750000
50%,38.244440,22.930860,257501.500000,0.920000,25184.000000
75%,39.636890,23.716220,665871.000000,1.500000,37866.000000
max,40.643610,25.143410,672130.000000,4.540000,145287.000000


## Visualization, Modeling, Machine Learning

Can you predict future demand and identify how different features influence that? Please explain your findings effectively to technical and non-technical audiences using comments and visualizations, if appropriate.
- **Build an optimized model that effectively solves the business problem.**
- **The model would be evaluated on the basis of mean absolute percent error.**
- **Read the Test.csv file and prepare features for testing.**

In [10]:
#Loading Test data
test_data=pd.read_csv('test.csv')
test_data.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price
0,6480,31/01/18,Athens,37.97945,23.71622,664046,shop_1,kinder-cola,plastic,1.5lt,3.10
1,6481,31/01/18,Athens,37.97945,23.71622,664046,shop_1,kinder-cola,can,330ml,0.85
2,6482,31/01/18,Athens,37.97945,23.71622,664046,shop_1,adult-cola,glass,500ml,0.83
3,6483,31/01/18,Athens,37.97945,23.71622,664046,shop_1,orange-power,glass,500ml,0.54
4,6484,31/01/18,Athens,37.97945,23.71622,664046,shop_1,orange-power,plastic,1.5lt,0.83




**The management wants to know what are the most important features for your model.  Can you tell them?**

> #### Task:
- **Visualize the top 20 features and their feature importance.**


> #### Task:
- **Submit the predictions on the test dataset using your optimized model.** <br/>
    For each record in the test set (`Test.csv`), you must predict the value of the `quantity` variable. You should submit a CSV file with a header row and one row per test entry. 
The file (`submissions.csv`) should have exactly 2 columns:
   - **id**
   - **quantity**

In [ ]:
#Submission
submission_df.to_csv('submissions.csv',index=False)

---